# Systems Programming

### Lecture 4: Makefiles and some more C

### Anne Reinarz

anne.k.reinarz@durham.ac.uk



## Recall: A First Program



In [ ]:
#include <stdio.h>

int main() {    
    printf("Hello, World!\n");    
    return 0;
}

Saved in a file with a "`.c`" file extension, for example
"`helloworld.c`"

# Compiling

![comp](comp.jpg)

# Compiling

## For now let's look at gcc

> gcc -o outfile file.c


-   Use `-o` to name the output
-   Use `-E` option to do pre-processing only, or call `cpp`

-   Use `-S` option to go as far as compilation only

-   Use `-c` option to go as far as assembly only

-   Use `nm` tool to investigate object libraries


# Compiling

## For now let's look at gcc

> gcc -o outfile file.c anotherfile.c -lboost -lncurses -I /some/path/


- Use `-l` to link e.g. external libraries
- Add more c files
- Add include path for more header files
- Our compiling process quickly becomes very complicated!


# Makefiles

- When we have a number of files to compile together we need a rule-set to perform this
- The `make` command provides this
- Requires a rule-file called the `Makefile`
- Declarative programming style set of rules for building the program

## Installing make

- Under linux no problem (try it on mira)
- On your own computer (if you use windows):
    - You can use the Windows subsystem for linux then you will have access to make and gcc as under linux (https://docs.microsoft.com/en-us/windows/wsl/install)
    - You can install it in VisualStudio: https://docs.microsoft.com/en-us/cpp/build/reference/creating-a-makefile-project?view=msvc-160

# Makefiles
- Format of each rule:

``target [target ...]: [component ...]
        [command 1]
        ...
        [command n]``

- N.B. Tab character
- `target` - what you want to make
- `component` - something which needs to exist (might need another rule)


# Makefiles: Example

- Let's say we have files:
    - `main.c`, `counter.h`, `counter.c`, `sales.h`,`sales.c`

In [ ]:
all: counter.o sales.o main.c
        gcc -o program main.c counter.o sales.o

counter.o: counter.c counter.h
        gcc -c counter.c

sales.o: sales.c sales.h
        gcc -c sales.c

clean:
        rm -rf program counter.o sales.o

## Makefiles: Macros

- Macros can be used to store definitions
    - `CC=gcc`

- They can be generated from commands
    - `DATE = date`


## Makefiles: Macros

- And used in the Makefile

In [ ]:
all:
        echo This was compiled using $(CC) on $(DATE)

- Running this gives:
    - This was compiled using gcc on Thu Oct 14 07:31:32 PM BST 2021

# Makefiles: Pattern Rules
- We can specify a pattern rule which matches multiple files
- e.g. compile C files into object files:

In [ ]:
DEPS = counter.h sales.h
%.o: %.c $(DEPS)
        gcc -c $< -o $@

# Makefiles: Pattern Rules

- contains the character ‘%’ (exactly one of them) in the target
- ‘%’ matches any nonempty substring (similar to * in bash)

### Example
- ‘%.c’ as a pattern matches any file name that ends in ‘.c’. 
- ‘s.%.c’ as a pattern matches any file name that starts with ‘s.’, ends in ‘.c’ and is at least five characters long. (There must be at least one character to match the ‘%’.)

# Makefiles: Automatic Variables

- How do you write a pattern rule? The name of file is different each time the implicit rule is applied.
- Solution: use automatic variables. These variables have values computed for each rule that is executed, based on the target
- The substring that the ‘%’ matches is called the stem




# Makefiles: Automatic Variables

- `$@`: The file name of the target of the rule. 
- `$<`: The name of the first prerequisite.

- `$?`: The names of all the prerequisites that are newer than the target, with spaces between them. 
- `$^`: The names of all the prerequisites, with spaces between them. Each is listed only once no matter how often it appears, to duplicate use `$+`
- `$*`:   The stem with which an implicit rule matches.
    - If the target is dir.a and the target pattern is %.a then the stem is dir.

This would change our original Makefile example to:

In [ ]:
CC=gcc
CFLAGS=-I.
DEPS = counter.h sales.h

all: counter.o sales.o main.c
        gcc -o program main.c counter.o sales.o
        
%.o: %.c $(DEPS)
	$(CC) -c -o $@ $< $(CFLAGS)
    
clean:
        rm -rf program counter.o sales.o

## What have we changed?
- Added CC, CFLAGS and DEPS allowing us to quickly change them
- first creates the macro DEPS, which is the set of .h files on which the .c files depend
- define a rule that applies to all files ending in  .o which says:
    - .o file depend on the .c version of the file and the .h files included in DEPS
- The `-c` flag tells the compiler to generate the object file
- the `-o $@` says to put the output of the compilation in the file named on the left side of the `:`
- the `$<` is the first item in the dependencies list

# Makefiles: A few comments
- Comments - lines starting with `#`
- Lazy evaluation
- If a target exists and has a timestamp later than all of its components assume it is up to date and don't bother to re-process
- Nothing to do with C: 
    - Although Makefiles are often used with C programs there is no intrinsic link - can use with any code/work
- You can run any specific rule by invoking its target:
    - `make sales.o`

# Beyond Make

- For very large project Makefiles become cumbersome. 
- They can automate some aspects of compilation, but not all
- They can't find and correctly link external libraries
- Many tools exist, most are difficult to set up but reasonably easy to use
- You may run into software that you have to install using these tools!

# Beyond Make

### Autotools

"The first goal of the Autotools is to simplify the development of portable programs. The system permits the developer to concentrate on writing the program, simplifying many details of portability across Unix and even Windows systems, and permitting the developer to describe how to build the program using simple rules rather than complex Makefiles."

- Not really very simple
- only works well under Linux
- used by a fair amount of open-source software

# Beyond Make

### CMake

"CMake is an open-source, cross-platform family of tools designed to build, test and package software. CMake is used to control the software compilation process using simple platform and compiler independent configuration files, and generate native makefiles and workspaces that can be used in the compiler environment of your choice."

- Very flexible
- Works best under Linux or MacOS
- Not always intuitive

# Beyond Make

### Less common options
 
- SCons:
    - A little newer than CMake and autotools
    - In the end: pretty similar in terms of usability
- Docker: 
    - Instead of having to install anything, many developers now just provide a docker image of their software
    - Advantage: Very easy on the user side
    - Essentially a light-weight VM

# Back to C

# Summary:
- In the first lecture we started writing a few simple programs in C
- We looked at primarily at the pre-processor
    - #include - for including header files
    - #define - to set constants
    - #ifdef, #endif - if statements
    - #define MAX(a,b) ((a)<(b)?(b):(a)) - a parameterised macro
- Now let's get down the basic syntax

# True / False and Comparison
- Traditionally, C did not have a boolean type and just uses `int`:
    -  0 - is false
    - Any other `int` is true
- Comparisons `<`, `<=`, `==`, `>=`, `>`, `!=` will evaluate to `1` if they hold and `0` if they don't


# True / False and Comparison


In [1]:
#include <stdio.h>

int main(){
    printf("Is 4<5? %d\n", 4<5);
    printf("Is 4==5? %d\n", 4==5);
    printf("Is 4!=5? %d\n", 4!=5);
    return 0;
}

Is 4<5? 1
Is 4==5? 0
Is 4!=5? 1


# True / False and Comparison

- (C99 introduced `bool`, which is defined in `stdbool.h`)
- You can still use the integer convention if you prefer

# Statements and Compound Statements

- A *statement* in C is a single instruction terminated with a semicolon
    - `printf("Hello world!\n");`


# Statements and Compound Statements

- A *compound statement* is a set of statements surrounded by curly brackets `{ }`
    - ``{
      printf("Hello ");
      printf("world!\n");
      }``

- You can always replace a statement with a compound statement
- C doesn't care about formatting -- but we need it!

# Iteration statements
C provides three iteration statements:

- The `while` statement is used for loops whose controlling expression is tested before the loop body is executed 

- The `do` statement is used if the expression is tested after the loop body is executed 

- The `for` statement is convenient for loops that increment or decrement a counting variable or iterator

# Iteration statements
### The `while` statement

- The while statement has the form:

``while ( expression ) {
    statement
   }``

- `expression` is the controlling expression
- `statement` is the loop body

- The expression is evaluated and if it is nonzero (true) then the loop body is executed
- The expression is tested before the loop body begins

### The `while` statement
### Example

In [2]:
#include <stdio.h>

int main(){
    int i = 8;
    while(i>0){
        printf("Hello planet\n");
        i = i-1;
    }
    return 0;
}

Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet


# Iteration statements
### The `do` statement

- The `do` statement has the form

``do {
    statement 
  }while ( expression );``

- `expression` is the controlling expression
- `statement` is the loop body

- The expression is evaluated and if it is nonzero (true) then the loop body is executed again
- The expression is tested after the loop body ends

### The `do` statement
### Example

In [3]:
#include <stdio.h>

int main(){
    int i = 8;
    do{
        printf("Hello planet\n");
        i = i-1;
    } while(i>0);
    return 0;
}

Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet


# Iteration statements
### The `for` statement

- The for statement is ideal for loops that have a counting variable, but it's versatile enough to be used for other kinds of loops as well


# Iteration statements
### The `for` statement

- General form of the `for` statement:

``for ( expr1 ; expr2 ; expr3 ){ 
    statement
  }``
 
- `expr1` - initialisation
- `expr2` - conditional
- `expr3` - increment



### The `for` statement
### Example


In [6]:
#include <stdio.h>

int main(){
    for(int i = 8; i >0; i=i-1){
        printf("Hello planet\n");
    }
    return 0;
}

Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet


# The `break` and `continue` statements

- `break;` - statement causes the innermost enclosing loop to be exited immediately


In [7]:
#include <stdio.h>

int main(){
    for(int i = 0; i < 8; i++){
        printf("Hello planet\n");
        break;
    }
    return 0;
}

Hello planet


# The `break` and `continue` statements

- `continue;` causes the next iteration for the loop to begin
- In the case of a `while` or `do` loop, the test part is executed immediately; 
- in the case of a `for` loop, control first passes to the increment step.

In [9]:
#include <stdio.h>

int main(){
    for(int i = 0; i < 8; i++){
        continue;
        printf("Hello planet\n");
    }
    return 0;
}

# The `if`-`else` statement

- `if` statement allows choice between two alternatives by testing an expression

- An `if` statement may have an `else` clause:
``if ( expr1 ) {
    statement1 
  }
  else {
    statement2
  }``

- When executed, `expr1` is evaluated;
    - if `expr1` is nonzero,`statement1` is executed 
    - otherwise `statement2` (if present) is executed 

### The `if`-`else` statement
### Example

In [12]:
#include <stdio.h>
#include <stdbool.h>

int main(){
    bool consider_pluto_a_planet = true;
    for(int i = 0; i < 9; i++){
        if(i==8){
            if(consider_pluto_a_planet){
                printf("Hello planet\n");
            }
            continue;
        }
        printf("Hello planet\n");
    }
    return 0;
}

Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet
Hello planet


# The `if`-`else` statement
### `if`-`else` ambiguity

- Technically C allows you to leave out braces, but:
- Sometimes hard to spot this in your code:

In [13]:
#include <stdio.h>

int main(){
  int n = 1;
  int s = -1;
  if (n > 0){
    for (int i = 0 ; i < n ; i++ ){
      if (s > 0){
        printf( "S %d \n", s );
      }
      else 
        printf("n less than zero\n");
      }
    }
    return 0;
}

n less than zero


- Adding the braces even where not strictly necessary can help
- As does indentation.


# Summary

- We looked at some basic C syntax
- 3 types of loops:
    - for
    - while
    - do
- continue, break
- if, else